In [2]:
import xgboost as xgb
import uproot 
import numpy as np
import coffea
from coffea.nanoaod import NanoEvents
from coffea.nanoaod.methods.leptons import *
from coffea import hist
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib.ticker import PercentFormatter
import pandas as pd

In [6]:
# events with 10mm displacement
fname_10mm="/uscms/home/ahall/nobackup/public/forTamas/iDM_nanoaod_ctau-10.root"
events_10mm = NanoEvents.from_file(fname_10mm,methods={"ElectronBPark":Electron})

#events with 1mm displacement
fname_1mm="/uscms/home/ahall/nobackup/public/forTamas/iDM_nanoaod.root"
events_1mm = NanoEvents.from_file(fname_1mm,methods={"ElectronBPark":Electron})

In [8]:
#event level MET-Jet cut
def MET_Jet_cut (events): 
    return events [
        (events.MET.pt >120)
        & (events.Jet.pt.max() > 80)
    ]

soft_eles_1mm = MET_Jet_cut(events_1mm).ElectronBPark
soft_eles_10mm = MET_Jet_cut(events_10mm).ElectronBPark

# recquirement for matched gen electrons
# as a cut
def matched_gen_cut (electrons):
    return electrons [
        (abs(electrons.matched_gen.pdgId) == 11) 
               & (electrons.matched_gen.parent.pdgId == 1000023)
               & (electrons.matched_gen.status == 1)
               & (electrons.matched_gen.hasFlags(["fromHardProcess", "isFirstCopy"]))
    ]
# as a mask
def matched_gen_mask (electrons):
    return ((abs(electrons.matched_gen.pdgId) == 11) 
               & (electrons.matched_gen.parent.pdgId == 1000023)
               & (electrons.matched_gen.status == 1) 
               & (electrons.matched_gen.hasFlags(["fromHardProcess", "isFirstCopy"])))

def gen_part_cut (genpart):
    return genpart [
        (abs(genpart.pdgId) == 11) 
               & (genpart.parent.pdgId == 1000023)
               & (genpart.status == 1) 
               & (genpart.hasFlags(["fromHardProcess", "isFirstCopy"]))
    ]

all_gen_eles_1mm = gen_part_cut(MET_Jet_cut(events_1mm).GenPart)
all_gen_eles_10mm = gen_part_cut(MET_Jet_cut(events_10mm).GenPart)

# define matched electrons
matched_eles_1mm = matched_gen_cut(soft_eles_1mm[soft_eles_1mm.isPF==False])
matched_eles_10mm = matched_gen_cut(soft_eles_10mm[soft_eles_10mm.isPF==False])

# defining unmatched electrons
unmatched_eles_1mm = soft_eles_1mm[(~(matched_gen_mask(soft_eles_1mm).fillna(False))) 
                                   & (soft_eles_1mm.isPF == False)]
unmatched_eles_10mm = soft_eles_10mm[(~(matched_gen_mask(soft_eles_10mm).fillna(False))) 
                                   & (soft_eles_10mm.isPF == False)]

In [46]:
def cut_pt(electrons):
    return electrons[
        (electrons.pt<5) 
        & (electrons.pt>1)
    ]

In [82]:
matched_var = 1 * matched_gen_mask(cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten()).fillna(False)
id_vars = {
    'dxy' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().dxy,
    'dz' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().dz,
    'ip3d' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().ip3d,
    'sip3d' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().sip3d,
    'fBrem' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().fBrem,
    'ptBiased' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().ptBiased,
    'mvaId' : cut_pt(soft_eles_1mm[soft_eles_1mm.isPF==False]).flatten().mvaId,
    'matched?' : matched_var
}
df = pd.DataFrame(id_vars)
df

,dxy,dz,ip3d,sip3d,fBrem,ptBiased,mvaId,matched?
0,0.000820,1.313477,1.282227,418.500000,0.942505,0.588714,0.316523,0
1,-0.028015,0.081726,0.038300,6.714844,0.169434,5.654523,5.772552,0
2,-0.002237,5.007812,3.263672,581.000000,-0.021198,-0.503604,-2.333933,0
3,-0.002035,0.572266,0.154175,32.156250,-0.017147,4.224657,-2.241945,0
4,0.006767,0.006096,0.007271,1.182617,0.666016,0.874815,1.248869,0
...,...,...,...,...,...,...,...,...
59282,-0.006897,2.773438,0.788574,74.187500,0.913696,-1.660778,0.659843,0
59283,-0.566406,-2.498047,0.866211,0.782227,0.034424,-1.109027,-1.370316,0
59284,-0.021957,2.800781,0.664551,57.062500,0.405640,-0.943574,1.422760,0
59285,0.033752,-1.981445,0.540527,54.718750,-0.103653,-0.979616,-3.351732,0
